In [ ]:
import pandas as pd
import numpy as np
from skimage.io import imread,imshow,imsave
from skimage.color import rgb2gray
import Functions as fc
import os
from skimage.filters.rank import mean
import matplotlib.pyplot as plt

In [ ]:
baseDeDados = './Imagens/DSFlowersV1/'

In [ ]:
flowers = fc.caminhos_dataset(baseDeDados)

In [ ]:
valores = [(7,1.0), (75,1.5), (77,2.0), (88,3.0)]

In [ ]:
for valor in valores:
    for flower in flowers:
        flowerCaminho = flower.replace(baseDeDados,"")  
        nomePasta1 = 'alargamento'+str(valor[0])+str(int(valor[1]))
        nomePasta2 = 'otsu'+'A'+str(valor[0])+str(int(valor[1]))
        # print(flowerCaminho)
        # print(nomePasta)
        caminho1,nomeFlower = fc.montaPasta(nomePasta1,flowerCaminho)
        caminho2, _ = fc.montaPasta(nomePasta2,flowerCaminho)
        if os.path.isdir(caminho1):
            imagem = fc.aplicaAlargamento(baseDeDados+flowerCaminho,valor[0],valor[1])
            imagem_otsu = fc.aplicaOtsu(imagem)
            plt.imsave(caminho1+nomeFlower,imagem,cmap=plt.cm.gray)
            plt.imsave(caminho2+nomeFlower,imagem_otsu,cmap=plt.cm.gray)
        else:
            os.makedirs(caminho1)
            os.makedirs(caminho2)
            imagem = fc.aplicaAlargamento(baseDeDados+flowerCaminho,valor[0],valor[1])
            plt.imsave(caminho1+nomeFlower,imagem,cmap=plt.cm.gray)
            plt.imsave(caminho2+nomeFlower,imagem_otsu,cmap=plt.cm.gray)